In [1]:
import pandas as pd
import re
import string
import sklearn
import csv

In [2]:
dataset_file="dataset_godfather.csv"
synopsis_file='synopsis_godfather.txt'
df = pd.read_csv(dataset_file)
file = open(synopsis_file,"r")
text=file.read()
sentences = text.split('. ')
df2 = pd.DataFrame(sentences, columns=["review_text"])
df2.to_csv('synopsis3.csv', index=False)
# print(df2.head())
df_pos=df.loc[df['is_spoiler'] == True]
df_neg=df.loc[df['is_spoiler'] == False]
pos_size=min(1000,df_pos.shape[0])
neg_size=min(500,df_neg.shape[0])
df_pos1=(df_pos.loc[:0.5*pos_size,:])
print(df_pos1.shape)
df_pos1.drop(["is_spoiler"],axis=1,inplace=True)
print(pos_size,neg_size)
train_size=0.5*(pos_size)
test_size=0.5*(pos_size+neg_size)
df_test_pos=df_pos.loc[0.5*pos_size+1:,:]
df_test_neg=df_neg.loc[:,:]
# df_train_neg.to_csv('train.csv', index=False)
print(df_pos1.shape)
frames_train=[df_pos1,df2]
frames_test=[df_test_pos,df_test_neg]
df_train=pd.concat(frames_train)
df_test=pd.concat(frames_test)
df_train.to_csv('train.csv', index=False)
df_test.to_csv('test.csv', index=False)

(211, 3)
420 500
(211, 2)


/home/shivashish/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/shivashish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
index=[]
for i in range(len(df_train.index)):
    index.append("Positive "+str(i))
#df.insert(loc=0,column='index',value=index)
df_train['index']=index
df_train.set_index('index', inplace=True)

index2=[]
for i in range(len(df_test.index)):
    index2.append("Comment "+str(i))
#df.insert(loc=0,column='index',value=index)
df_test['index']=index2
df_test.set_index('index', inplace=True)
df_test

,Unnamed: 0,is_spoiler,review_text
index,,,
Comment 0,4572,True,This film follows Vito Corleone the leader of ...
Comment 1,4573,True,"The superb, three-part gangster saga was inaug..."
Comment 2,4574,True,Rarely can it be said that a film has defined ...
Comment 3,4575,True,This movie has everything a masterpiece needs....
Comment 4,4576,True,"I must confess, I find it really tricky to wat..."
Comment 5,4577,True,The Godfather is a 1972 masterpiece. It is bas...
Comment 6,4578,True,The Godfather is a great movie. From it's writ...
Comment 7,4579,True,The Godfather is quite simply the best film of...
Comment 8,4580,True,I'd been putting off watching The Godfather fo...


In [4]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
round1 = lambda x: clean_text_round1(x)

In [5]:
data_clean = pd.DataFrame(df_train.review_text.apply(round1))
data_clean2 = pd.DataFrame(df_test.review_text.apply(round1))
data_clean2

,review_text
index,
Comment 0,this film follows vito corleone the leader of ...
Comment 1,the superb threepart gangster saga was inaugur...
Comment 2,rarely can it be said that a film has defined ...
Comment 3,this movie has everything a masterpiece needs ...
Comment 4,i must confess i find it really tricky to watc...
Comment 5,the godfather is a masterpiece it is based on...
Comment 6,the godfather is a great movie from its writin...
Comment 7,the godfather is quite simply the best film of...
Comment 8,id been putting off watching the godfather for...


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.review_text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index


cv2 = CountVectorizer(stop_words='english')
data_cv2 = cv2.fit_transform(data_clean2.review_text)
data_dtm2 = pd.DataFrame(data_cv2.toarray(), columns=cv2.get_feature_names())
data_dtm2.index = data_clean2.index
data_dtm2

,aback,abandando,abandon,abandoning,abduction,abe,abhorrence,abiding,abilities,ability,...,zoom,zoomin,zooming,zooms,zootrope,ángel,ýts,шit,尷尬,最重要的
index,,,,,,,,,,,,,,,,,,,,,
Comment 0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Comment 3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
import numpy as np
from gensim import matutils, models
import scipy.sparse

In [12]:
id_word_dict={}
word_id_dict={}
word_dict={}
word_dict2={}
temp_set = set(data_dtm.columns)
l=0
for i in data_dtm.columns:
    word_dict[i]=l
    l+=1
l=0
for i in data_dtm2.columns:
    word_dict2[i]=l
    l+=1
    temp_set.add(i)
j=0
for word in temp_set:
    id_word_dict[j]=word
    word_id_dict[word]=j
    j+=1

In [13]:
no_of_words = len(id_word_dict)
train_size = len(df_train.index)
test_size = len(df_test.index)
X_train = np.zeros((train_size,no_of_words))
X_test = np.zeros((test_size,no_of_words))
Y_test = np.zeros(test_size)

In [19]:
for i in range(train_size):
    s = set(data_clean.iloc[i][0].split(' '))
    for wor in s:
        if wor in temp_set:
            j = word_dict[wor]
            index = word_id_dict[wor]
            X_train[i][index]+=data_dtm.iloc[i][j]

In [20]:
for i in range(test_size):
    s = set(data_clean2.iloc[i][0].split(' '))
    for wor in s:
        if wor in temp_set:
            j = word_dict2[wor]
            index = word_id_dict[wor]
            X_test[i][index]+=data_dtm2.iloc[i][j]

In [21]:
for i in range(test_size):
    Y_test[i]=df_test.iloc[i]["is_spoiler"]

In [22]:
Y_test

array([1., 1., 1., ..., 0., 0., 0.])

In [23]:
from sklearn.svm import OneClassSVM
lpu_model = OneClassSVM(gamma='auto').fit(X_train)
Y_out = lpu_model.predict(X_test)

In [24]:
wrong=0
falsePositive=0
falseNegative=0
truePositive=0
trueNegative=0
for i in range(len(Y_out)):
    if Y_out[i]==1 and Y_test[i]==1:
        truePositive+=1
    elif Y_out[i]==-1 and Y_test[i]==0:
        trueNegative+=1    
    elif Y_out[i]==1 and Y_test[i]==0:
        falsePositive+=1
        wrong+=1
    elif Y_out[i]==-1 and Y_test[i]==1:
        falseNegative+=1
        wrong+=1

print(wrong/len(Y_out))
print(falseNegative)
print(falsePositive)
print(truePositive)
print(trueNegative)
precision=truePositive/(truePositive+falsePositive)
recall=truePositive/(truePositive+falseNegative)
f1score=(2*precision*recall)/(precision+recall)
print(f1score)

0.887331256490135
9
1700
200
17
0.1896633475580844
